# Screenshot 1

**Goal:** Set up simple environment detection (DEV/PROD), define base folders, create output/log directories,
and prepare daily log/listing files. This mirrors the SAS macro setup you shared.

> Tip: Run this cell top-to-bottom before other sections. It's intentionally simple and readable.


In [ ]:
# --- Screenshot 1: Environment & Paths (Beginner-Friendly) ---
# Keep this cell simple. It's meant to be clear for a new engineer.
from pathlib import Path
import os
import socket
from datetime import datetime
import logging
import pandas as pd

# 1) Detect environment: DEV or PROD (very simple heuristic)
user = (os.getenv("USER") or os.getenv("USERNAME") or "").strip()
host = socket.gethostname().lower()

# If username starts with 'U' and host is not 'usasst11', treat it as PROD (mimics the SAS logic)
env = "PROD" if (user[:1].upper() == "U" and host != "usasst11") else "DEV"

# 2) Base path similar to SAS &regpath
#    Adjust these if your org uses different mount points.
regpath = "/sas/RSD/REG" if env == "PROD" else "/sas/RSD/REG_DEV"

# 3) Output folders (similar to &logpath and &outpath in SAS)
logpath = Path(regpath) / "CB6" / "log" / "product_appropriateness" / "client360"
outpath = Path(regpath) / "output" / "product_appropriateness" / "client360"

# Make sure directories exist (no-op if they already do)
logpath.mkdir(parents=True, exist_ok=True)
outpath.mkdir(parents=True, exist_ok=True)

# 4) Daily filenames (yyyymmdd) like SAS &ymd and files
ymd = datetime.now().strftime("%Y%m%d")
logfile = logpath / f"CB6_pa_client360_{ymd}.log"
lstfile = logpath / f"CB6_pa_client360_{ymd}.lst"   # SAS "listing" file; we just create a simple text file

# 5) Minimal logging setup (beginner-friendly)
logging.basicConfig(
    filename=str(logfile),
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s"
)
logging.info("Starting run...")
logging.info("User=%s Host=%s Env=%s", user, host, env)
logging.info("regpath=%s", regpath)
logging.info("logpath=%s", str(logpath))
logging.info("outpath=%s", str(outpath))

# Also write a simple header line to the listing file to mimic PROC PRINTTO behavior
with open(lstfile, "a", encoding="utf-8") as f:
    f.write(f">>> Starting Running Time - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Environment: {env}\n")
    f.write(f"User: {user}  Host: {host}\n")
    f.write(f"regpath: {regpath}\n")
    f.write("\n")

# 6) Capture the run configuration in a tiny DataFrame (so we’re using dataframes from the start)
run_config = pd.DataFrame([{
    "user": user,
    "host": host,
    "env": env,
    "regpath": regpath,
    "logpath": str(logpath),
    "outpath": str(outpath),
    "logfile": str(logfile),
    "lstfile": str(lstfile),
    "run_datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
}])

# Display the configuration so it's visible in the notebook
run_config
